In [2]:
import torch

from pathlib import Path

from app.model.config import Config
from app.model.time_series_embedding import TimeSeriesEmbedding

data_dir = Path('app/data')
config = Config(data_dir)
time_series_embedding = TimeSeriesEmbedding(config)
input = torch.randn(32, config.seq_len, config.input_dims)
output = time_series_embedding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 5]); output.shape=torch.Size([32, 5, 150])'

In [3]:
import torch
from app.model.config import Config
from app.model.positional_encoding import PositionalEncoding

data_dir = Path('app/data')
config = Config(data_dir)
positional_encoding = PositionalEncoding(config)
input = torch.randn((32, config.seq_len, config.d_model))
output = positional_encoding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 150]); output.shape=torch.Size([32, 5, 150])'

In [4]:
import torch
from app.model.config import Config
from app.model.stock_transformer import StockTransformer


data_dir = Path('app/data')
config = Config(data_dir)
input = torch.randn((config.batch_size, config.seq_len, config.input_dims),
                    dtype=torch.float)
display(f'{input.shape=}')
transformer = StockTransformer(config)
output = transformer(input)
display(f'{output.shape=}')

'input.shape=torch.Size([16, 5, 5])'

'output.shape=torch.Size([16, 1])'

In [ ]:
import csv
import yfinance as yf

from pathlib import Path


ticker = 'SPY'
data = yf.Ticker(ticker)
hist = data.history(period='36mo', interval='1d')

if hist.empty:
    print(f'No data available for {ticker}')
else:
    date_from = hist.index.min().strftime('%Y%m%d')
    date_to = hist.index.max().strftime('%Y%m%d')

    out_dir = Path('app/data') / 'tickers'
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{ticker}_{date_from}_{date_to}.csv"

    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    with out_path.open('w', newline='') as fp:
        writer = csv.writer(fp)
        writer.writerow(columns)
        for row in hist.itertuples():
            writer.writerow([getattr(row, c) for c in columns])

    print(f'Saved {out_path}')

Saved app/data/tickers/SPY_20230103_20251231.csv


In [3]:
import torch
from pathlib import Path

from app.model.checkpointer import Checkpointer


model_path = Path('app/data/models/5000/SPY.pkl')
model, config, step = Checkpointer.load_checkpoint(model_path)
input = torch.randn((1, config.seq_len, config.input_dims))
output = model(input)
display(input, output)

Model loaded from app/data/models/5000/SPY.pkl


tensor([[[ 0.2452, -1.0715, -0.8595,  1.0232, -0.2170,  0.2105],
         [ 0.2699,  1.9293, -0.3124, -0.0569, -0.7479,  0.1794],
         [ 1.9890,  0.0382,  0.6558,  0.8074,  0.2845,  0.3270],
         [ 0.3887, -2.2356,  0.7075,  0.8950, -1.1945,  0.4842],
         [ 1.5931, -0.3992,  0.3223, -0.7252, -1.3187,  0.9755]]])

tensor([[0.0029]], grad_fn=<AddmmBackward0>)